# Olist Data Processing and RAG System

This notebook processes the Olist merged CSV file, creates unique order IDs, builds a relational database, and generates vector embeddings for RAG.


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import hashlib
from tqdm import tqdm

# Load the CSV file
print("Loading CSV file...")
df = pd.read_csv('olist_full_merged.csv', low_memory=False)
print(f"Loaded {len(df)} rows with {len(df.columns)} columns")
print(f"Columns: {list(df.columns)}")


In [ ]:
# Create unique order_id based on order_id + customer_id + seller_id combination
# If two rows have the same combination, they get the same unique ID
print("Creating unique order IDs...")

# Create a combination key
df['order_customer_seller_key'] = (
    df['order_id'].astype(str) + '_' + 
    df['customer_id'].astype(str) + '_' + 
    df['seller_id'].astype(str)
)

# Create a mapping of unique combinations to sequential IDs
unique_combinations = df['order_customer_seller_key'].unique()
combination_to_id = {combo: idx + 1 for idx, combo in enumerate(unique_combinations)}

# Map the new unique IDs
df['unique_order_id'] = df['order_customer_seller_key'].map(combination_to_id)

# Replace the original order_id with the unique one
df['order_id'] = df['unique_order_id']
df = df.drop(['order_customer_seller_key', 'unique_order_id'], axis=1)

print(f"Created {len(unique_combinations)} unique order IDs")
print(f"Sample of new order_ids: {df['order_id'].head(10).tolist()}")


In [ ]:
# Save the processed CSV
print("Saving processed CSV...")
df.to_csv('olist_processed.csv', index=False)
print("Saved to olist_processed.csv")


In [ ]:
# Create relational database
print("Creating relational database...")

# Connect to SQLite database
conn = sqlite3.connect('olist_database.db')
cursor = conn.cursor()

# Drop existing tables if they exist (for re-runs)
print("Clearing existing tables...")
tables = ['reviews', 'payments', 'order_items', 'products', 'sellers', 'customers', 'orders']
for table in tables:
    cursor.execute(f'DROP TABLE IF EXISTS {table}')
conn.commit()

# Create tables based on the data structure
# Orders table
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_customer_date TEXT,
    order_delivered_carrier_date TEXT,
    order_estimated_delivery_date TEXT,
    delivery_time_days REAL
)
''')

# Customers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_city TEXT,
    customer_state TEXT,
    customer_zip_code_prefix TEXT,
    customer_lat REAL,
    customer_lng REAL
)
''')

# Sellers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS sellers (
    seller_id TEXT PRIMARY KEY,
    seller_city TEXT,
    seller_state TEXT,
    seller_zip_code_prefix TEXT,
    seller_lat REAL,
    seller_lng REAL
)
''')

# Products table
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_category_name_english TEXT,
    product_name_lenght REAL,
    product_description_lenght REAL,
    product_photos_qty REAL,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL
)
''')

# Order items table
cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items (
    order_id INTEGER,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    customer_id TEXT,
    price REAL,
    freight_value REAL,
    item_total_value REAL,
    shipping_limit_date TEXT,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (seller_id) REFERENCES sellers(seller_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)
''')

# Payments table
cursor.execute('''
CREATE TABLE IF NOT EXISTS payments (
    order_id INTEGER,
    payment_types TEXT,
    total_payment_value REAL,
    payment_count REAL,
    order_total_value REAL,
    PRIMARY KEY (order_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
)
''')

# Reviews table
cursor.execute('''
CREATE TABLE IF NOT EXISTS reviews (
    order_id INTEGER,
    avg_review_score REAL,
    review_count REAL,
    review_comments TEXT,
    review_emotion TEXT,
    review_emotion_intensity REAL,
    PRIMARY KEY (order_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
)
''')

conn.commit()
print("Database schema created successfully")


In [ ]:
# Populate the database tables
print("Populating database tables...")

# Extract unique records for each table
print("Extracting unique orders...")
orders_df = df[['order_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_customer_date', 'order_delivered_carrier_date', 
                'order_estimated_delivery_date', 'delivery_time_days']].drop_duplicates(subset=['order_id'])
orders_df.to_sql('orders', conn, if_exists='replace', index=False)

print("Extracting unique customers...")
customers_df = df[['customer_id', 'customer_unique_id', 'customer_city', 'customer_state', 
                   'customer_zip_code_prefix', 'customer_lat', 'customer_lng']].drop_duplicates(subset=['customer_id'])
customers_df.to_sql('customers', conn, if_exists='replace', index=False)

print("Extracting unique sellers...")
sellers_df = df[['seller_id', 'seller_city', 'seller_state', 'seller_zip_code_prefix', 
                'seller_lat', 'seller_lng']].drop_duplicates(subset=['seller_id'])
sellers_df.to_sql('sellers', conn, if_exists='replace', index=False)

print("Extracting unique products...")
products_df = df[['product_id', 'product_category_name', 'product_category_name_english', 
                  'product_name_lenght', 'product_description_lenght', 'product_photos_qty',
                  'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']].drop_duplicates(subset=['product_id'])
products_df.to_sql('products', conn, if_exists='replace', index=False)

print("Extracting order items...")
order_items_df = df[['order_id', 'order_item_id', 'product_id', 'seller_id', 'customer_id',
                     'price', 'freight_value', 'item_total_value', 'shipping_limit_date']]
order_items_df.to_sql('order_items', conn, if_exists='replace', index=False)

print("Extracting payments...")
payments_df = df[['order_id', 'payment_types', 'total_payment_value', 'payment_count', 
                  'order_total_value']].drop_duplicates(subset=['order_id'])
payments_df.to_sql('payments', conn, if_exists='replace', index=False)

print("Extracting reviews...")
reviews_df = df[['order_id', 'avg_review_score', 'review_count', 'review_comments', 
                 'review_emotion', 'review_emotion_intensity']].drop_duplicates(subset=['order_id'])
reviews_df.to_sql('reviews', conn, if_exists='replace', index=False)

conn.commit()
print("Database populated successfully!")


In [ ]:
# Verify database
cursor.execute("SELECT COUNT(*) FROM orders")
print(f"Orders in database: {cursor.fetchone()[0]}")

cursor.execute("SELECT COUNT(*) FROM customers")
print(f"Customers in database: {cursor.fetchone()[0]}")

cursor.execute("SELECT COUNT(*) FROM sellers")
print(f"Sellers in database: {cursor.fetchone()[0]}")

cursor.execute("SELECT COUNT(*) FROM products")
print(f"Products in database: {cursor.fetchone()[0]}")

cursor.execute("SELECT COUNT(*) FROM order_items")
print(f"Order items in database: {cursor.fetchone()[0]}")

conn.close()
print("Database connection closed")


## Vector Embeddings Generation

Now we'll create vector embeddings for all columns to enable RAG functionality.


In [ ]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
from tqdm import tqdm
import json
import re

# Load the processed data
print("Loading processed data...")
df = pd.read_csv('olist_processed.csv', low_memory=False)

# Initialize the embedding model
print("Loading embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and effective model
print("Model loaded successfully")

# Initialize emotion extraction using a simple Portuguese sentiment lexicon
print("Setting up emotion extraction for Portuguese reviews...")

# Portuguese emotion/sentiment keywords
positive_words = [
    'perfeito', 'excelente', 'ótimo', 'bom', 'satisfeito', 'gostei', 'adorei', 'recomendo',
    'qualidade', 'rápido', 'antes do prazo', 'surpreendeu', 'muito bom', 'top', 'show',
    'maravilhoso', 'incrível', 'fantástico', 'amor', 'feliz', 'content', 'satisfeito'
]

negative_words = [
    'ruim', 'péssimo', 'horrível', 'decepcionado', 'não gostei', 'problema', 'defeito',
    'quebrado', 'danificado', 'atrasado', 'demorado', 'triste', 'chateado', 'insatisfeito',
    'não recebi', 'não veio', 'faltou', 'errado', 'diferente', 'avariação', 'reclamação'
]

neutral_words = [
    'ok', 'normal', 'regular', 'aceitável', 'básico', 'esperado'
]

def extract_emotion_from_review(review_text):
    """Extract emotion from Portuguese review text"""
    if pd.isna(review_text) or not str(review_text).strip():
        return 'neutral', 0.0
    
    review_lower = str(review_text).lower()
    
    # Count positive and negative words
    positive_count = sum(1 for word in positive_words if word in review_lower)
    negative_count = sum(1 for word in negative_words if word in review_lower)
    
    # Check for emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    
    emojis = emoji_pattern.findall(review_text)
    positive_emojis = ['😊', '😍', '😁', '😄', '😃', '👍', '❤️', '💯', '⭐', '🌟', '✨']
    negative_emojis = ['😢', '😞', '😠', '😡', '👎', '😥', '😔']
    
    emoji_score = 0
    for emoji in emojis:
        if any(pe in emoji for pe in positive_emojis):
            emoji_score += 1
        elif any(ne in emoji for ne in negative_emojis):
            emoji_score -= 1
    
    # Calculate sentiment score
    sentiment_score = positive_count - negative_count + (emoji_score * 0.5)
    
    # Determine emotion
    if sentiment_score > 1:
        emotion = 'positive'
        intensity = min(1.0, abs(sentiment_score) / 5.0)
    elif sentiment_score < -1:
        emotion = 'negative'
        intensity = min(1.0, abs(sentiment_score) / 5.0)
    else:
        emotion = 'neutral'
        intensity = 0.5
    
    return emotion, intensity

# Extract emotions from reviews
print("Extracting emotions from reviews...")
emotions = []
intensities = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting emotions"):
    emotion, intensity = extract_emotion_from_review(row.get('review_comments', ''))
    emotions.append(emotion)
    intensities.append(intensity)

df['review_emotion'] = emotions
df['review_emotion_intensity'] = intensities

print(f"Emotion distribution:")
print(df['review_emotion'].value_counts())
print("\nEmotion extraction complete!")


In [ ]:
# Create comprehensive text representations for each row using ALL columns
print("Creating text representations from all columns...")

def create_comprehensive_text(row):
    """Create a comprehensive text representation using all columns"""
    text_parts = []
    
    # Order information
    text_parts.append(f"Order ID: {row['order_id']}")
    text_parts.append(f"Order Status: {row['order_status']}")
    text_parts.append(f"Purchase Date: {row['order_purchase_timestamp']}")
    text_parts.append(f"Approved Date: {row['order_approved_at']}")
    text_parts.append(f"Delivery Date: {row['order_delivered_customer_date']}")
    text_parts.append(f"Estimated Delivery: {row['order_estimated_delivery_date']}")
    text_parts.append(f"Delivery Time: {row['delivery_time_days']} days")
    
    # Customer information
    text_parts.append(f"Customer ID: {row['customer_id']}")
    text_parts.append(f"Customer Unique ID: {row['customer_unique_id']}")
    text_parts.append(f"Customer City: {row['customer_city']}")
    text_parts.append(f"Customer State: {row['customer_state']}")
    text_parts.append(f"Customer Zip: {row['customer_zip_code_prefix']}")
    text_parts.append(f"Customer Location: ({row['customer_lat']}, {row['customer_lng']})")
    
    # Seller information
    text_parts.append(f"Seller ID: {row['seller_id']}")
    text_parts.append(f"Seller City: {row['seller_city']}")
    text_parts.append(f"Seller State: {row['seller_state']}")
    text_parts.append(f"Seller Zip: {row['seller_zip_code_prefix']}")
    text_parts.append(f"Seller Location: ({row['seller_lat']}, {row['seller_lng']})")
    
    # Product information
    text_parts.append(f"Product ID: {row['product_id']}")
    text_parts.append(f"Product Category: {row['product_category_name']} ({row['product_category_name_english']})")
    text_parts.append(f"Product Name Length: {row['product_name_lenght']}")
    text_parts.append(f"Product Description Length: {row['product_description_lenght']}")
    text_parts.append(f"Product Photos: {row['product_photos_qty']}")
    text_parts.append(f"Product Weight: {row['product_weight_g']}g")
    text_parts.append(f"Product Dimensions: {row['product_length_cm']}cm x {row['product_width_cm']}cm x {row['product_height_cm']}cm")
    
    # Order item information
    text_parts.append(f"Order Item ID: {row['order_item_id']}")
    text_parts.append(f"Price: R$ {row['price']}")
    text_parts.append(f"Freight Value: R$ {row['freight_value']}")
    text_parts.append(f"Item Total: R$ {row['item_total_value']}")
    text_parts.append(f"Shipping Limit: {row['shipping_limit_date']}")
    
    # Payment information
    text_parts.append(f"Payment Type: {row['payment_types']}")
    text_parts.append(f"Total Payment: R$ {row['total_payment_value']}")
    text_parts.append(f"Order Total: R$ {row['order_total_value']}")
    text_parts.append(f"Payment Count: {row['payment_count']}")
    
    # Review information
    text_parts.append(f"Review Score: {row['avg_review_score']}")
    text_parts.append(f"Review Count: {row['review_count']}")
    if pd.notna(row['review_comments']) and str(row['review_comments']).strip():
        text_parts.append(f"Review Comment: {row['review_comments']}")
    
    # Emotion information
    if 'review_emotion' in row:
        text_parts.append(f"Review Emotion: {row['review_emotion']}")
        text_parts.append(f"Emotion Intensity: {row['review_emotion_intensity']:.2f}")
    
    return " | ".join([str(part) for part in text_parts if pd.notna(part)])

# Create text representations
print("Generating text representations...")
texts = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    text = create_comprehensive_text(row)
    texts.append(text)

print(f"Created {len(texts)} text representations")
print(f"Sample text length: {len(texts[0])} characters")


In [ ]:
# Generate embeddings in batches
print("Generating vector embeddings...")
batch_size = 100
embeddings = []

for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, show_progress_bar=False)
    embeddings.append(batch_embeddings)

embeddings = np.vstack(embeddings)
print(f"Generated embeddings shape: {embeddings.shape}")

# Save embeddings and metadata
print("Saving embeddings...")
np.save('embeddings.npy', embeddings)

# Save text representations and corresponding row indices
metadata = {
    'texts': texts,
    'row_indices': list(range(len(texts)))
}

with open('embeddings_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print("Embeddings saved successfully!")


In [ ]:
# Create a vector store index using FAISS for efficient similarity search
try:
    import faiss
    
    print("Creating FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 distance
    
    # Normalize embeddings for cosine similarity (optional but often better)
    faiss.normalize_L2(embeddings)
    index.add(embeddings.astype('float32'))
    
    # Save the index
    faiss.write_index(index, 'embeddings_index.faiss')
    print("FAISS index created and saved!")
    
except ImportError:
    print("FAISS not available, using numpy-based search instead")
    # We'll use numpy-based search in the RAG app if FAISS is not available
